In [4]:
# f'''=HYPERLINK("{link}","Link")'''

'=HYPERLINK("https://web.bcpa.net/bcpaclient/#/Record-Search?fnumber=484125280260","Link")'

In [15]:
from typing import List
import requests
import json
import pandas as pd

import time
from tqdm import tqdm
def reverse(text:List):
    return ",".join(text[::-1])
import pymongo

client = pymongo.MongoClient("mongodb://127.0.0.1:27017")

db = client["bcpa"]
correct = db["correct"]
wrong = db["wrong"]
wrong_again=db['wrong2']
wrong_three=db['wrong3']

def find_index_by_owner_name(data, owner_name):
    for i, item in enumerate(data):
        if item.get('ownerName1') == owner_name:
            return i
    return -1  # Owner name not found


def change_order(text:str)->str:
    firstname=text.split(",")[0]
    secondname=text.split(",")[-1].split(" ")[0].strip()
    return ",".join([firstname,secondname])

In [30]:
# get the wrong data now
owner_names=[x['name'] for x in wrong_three.find()]
correct_fuzzy=db['correct_fuzzy']
wrong_final=db['wrong_final']

# now do fuzzy match
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_match(input_list,name):
    # Use the process.extractOne() function to find the closest match
    closest_match = process.extractOne(name, input_list)

    # closest_match is a tuple containing the closest match and its similarity score
    try:
        matched_name = closest_match[0]
        return matched_name
    except TypeError:
        return None
def populate_latest(x)->List:
    url = "https://web.bcpa.net/bcpaclient/search.aspx/PopulateInput"

    payload = json.dumps({"value": x,"cities":"", "arrayOfValues": ""})
    headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json; charset=UTF-8',
    'Origin': 'https://web.bcpa.net',
    'Referer': 'https://web.bcpa.net/bcpaclient/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    data_to_check=response.json()['d']
    return data_to_check


In [29]:


headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json; charset=UTF-8',
    'Origin': 'https://web.bcpa.net',
    'Referer': 'https://web.bcpa.net/BcpaClient/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

links=[]
error_name=[]
def get_url(name:str):
        time.sleep(2)
        close_check=populate_latest(name)
        close=fuzzy_match(close_check,name)
        time.sleep(2)
        if close:

            data = {"value":close ,"cities": "","orderBy": "NAME","pageNumber":"1","pageCount":"5000","arrayOfValues":"", "selectedFromList": "true","totalCount":"Y"}
            data=json.dumps(data)
            response = requests.post('https://web.bcpa.net/BcpaClient/search.aspx/GetData', headers=headers, data=data)
            data=response.json()
            data_to_check=data['d']['resultListk__BackingField']
            index=0
            try:
                dname=data_to_check[index]['folioNumber']
                link=f"https://web.bcpa.net/bcpaclient/#/Record-Search?fnumber={dname}"
                correct_fuzzy.insert_one({"name":name,"link":link})
                wrong_again.delete_one({"name":name})
            except IndexError:
                wrong_final.insert_one({"name":name})
        else:
             wrong_final.insert_one({"name":name})
             

In [20]:
for owner in tqdm(range(0,len((owner_names)))):

    get_url(owner_names[owner])
    

100%|██████████| 1194/1194 [1:26:31<00:00,  4.35s/it]


owner_names

  0%|          | 0/507 [00:02<?, ?it/s]


NameError: name 'populate_latest' is not defined